In [5]:
import json
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import os

# Get the list of files in the folder
testing_files = os.listdir('../datasets/New/Thumb')
training_files = os.listdir('../datasets/New/HandClose')

data = []
num_classes = 2
obj = {
    "HandClose": 0,
    "Thumb": 1,
    "Index": 2,
    "Middle": 3,
    "Ring": 4,
    "Pinky": 5
}

#Testing Data
for file in testing_files:
    d = {}
    with open(f'../datasets/New/Thumb/{file}', 'r') as f:
        l = [int(line.strip()) for line in f.readlines()]
        d['data'] = l
        d['label'] = "Thumb"
        data.append(d)
#print(len(testing_data))

#Training Data
for file in training_files:
    d = {}
    with open(f'../datasets/New/HandClose/{file}', 'r') as f:
        l = [int(line.strip()) for line in f.readlines()]
        d['data'] = l
        d['label'] = "HandClose"
        data.append(d)
#print(len(training_data))

x_data = [[int(x) for x in d['data']] for d in data]
y_data_c = [obj[d['label']] for d in data]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_data_norm = scaler.fit_transform(x_data)

x_data = np.array(x_data_norm)
y_data = np.array(y_data_c)
y_data = tf.keras.utils.to_categorical(y_data, num_classes)

# define the number of folds
k = 5

# create a KFold object
kf = KFold(n_splits=k, shuffle=True)

# iterate over the folds
for fold, (train_index, test_index) in enumerate(kf.split(x_data)):
    # get the training and test data for this fold
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    # normalize the data
    x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
    x_test = (x_test - x_train.min()) / (x_train.max() - x_train.min())

    dim = 58
    drpt = 0.2
    
    # create the model
    model = Sequential([
        tf.keras.layers.Dense(dim, activation='relu', input_shape=(499,)),
        tf.keras.layers.Dropout(drpt),
        tf.keras.layers.Dense(dim, activation='relu'),
        tf.keras.layers.Dropout(drpt),
        tf.keras.layers.Dense(dim, activation='relu'),
        tf.keras.layers.Dropout(drpt),
        tf.keras.layers.Dense(dim, activation='relu'),
        tf.keras.layers.Dropout(drpt),
        tf.keras.layers.Dense(dim, activation='relu'),
        #tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(dim, activation='relu'),
        #tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(num_classes, activation='softmax'),
    ])

    # compile the model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    # train the model on the training data
    model.fit(x_train, y_train, epochs=499, batch_size=499, verbose=0)

    # evaluate the model on the test data
    loss, accuracy = model.evaluate(x_test, y_test, batch_size=499, verbose=0)

    # print the test accuracy for this fold
    print(f'Fold {fold+1} Test accuracy: {accuracy:.4f}')
    
import pickle
Pkl_Filename = "Pickle_NN_Model_499.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

Fold 1 Test accuracy: 0.4737
Fold 2 Test accuracy: 0.5789
Fold 3 Test accuracy: 0.5263
Fold 4 Test accuracy: 0.5405
Fold 5 Test accuracy: 0.4865
